# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm as tqdm_notebook
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error as mse



# Data Preperation

```
# Exploratory Data Analysis
```



In [ ]:
df = pd.read_csv("household_power_consumption_household_power_consumption.csv")

In [ ]:
df.head()
df.shape #9 columns

(260640, 9)

In [ ]:
df.head(20)

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,1/1/07,0:00:00,2.58,0.136,241.97,10.6,0,0,0.0
1,1/1/07,0:01:00,2.552,0.1,241.75,10.4,0,0,0.0
2,1/1/07,0:02:00,2.55,0.1,241.64,10.4,0,0,0.0
3,1/1/07,0:03:00,2.55,0.1,241.71,10.4,0,0,0.0
4,1/1/07,0:04:00,2.554,0.1,241.98,10.4,0,0,0.0
5,1/1/07,0:05:00,2.55,0.1,241.83,10.4,0,0,0.0
6,1/1/07,0:06:00,2.534,0.096,241.07,10.4,0,0,0.0
7,1/1/07,0:07:00,2.484,0,241.29,10.2,0,0,0.0
8,1/1/07,0:08:00,2.468,0,241.23,10.2,0,0,0.0
9,1/1/07,0:09:00,2.486,0,242.18,10.2,0,0,0.0


In [ ]:
#Shifting Powers to the last column 
global_active_power = df.pop('Global_active_power')
global_reactive_power = df.pop('Global_reactive_power')
df['Global_active_power'] = global_active_power
df['Global_reactive_power'] = global_reactive_power #adding them tot he end of the result

In [ ]:
# df.dtypes
df.infer_objects().dtypes

Date                      object
Time                      object
Global_active_power       object
Global_reactive_power     object
Voltage                   object
Global_intensity          object
Sub_metering_1            object
Sub_metering_2            object
Sub_metering_3           float64
dtype: object

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df['DateTime'] = pd.to_datetime(df['Date'].dt.strftime('%Y-%m-%d') + ' ' + df['Time'])

df.set_index('DateTime', inplace=True)

df.drop(['Date', 'Time'], axis=1, inplace=True)

In [ ]:
#Object to numeric datatype
for i in range(len(df.columns)):
    try:
      df[df.columns[i]] = pd.to_numeric(df[df.columns[i]], errors='coerce')
    except Exception as e:
      pass

In [ ]:
df.infer_objects().dtypes

In [ ]:
tempV = df['Voltage']
tempV.plot()

In [ ]:
tempP = df['Global_active_power']
tempP.plot()

In [ ]:
tempS = df['Sub_metering_1']
tempS.plot()

In [ ]:
tempS2 = df['Sub_metering_2']
tempS.plot()

In [ ]:
tempS3 = df['Sub_metering_3']
tempS.plot()

In [ ]:
df_group_1 = df[['Voltage',	'Global_intensity',	'Sub_metering_1', 'Global_active_power','Global_reactive_power']]
df_group_2 = df[['Voltage',	'Global_intensity',	'Sub_metering_2', 'Global_active_power','Global_reactive_power']]
df_group_3 = df[['Voltage',	'Global_intensity',	'Sub_metering_3', 'Global_active_power','Global_reactive_power',]]

In [ ]:
# df_sub_metering_1.describe()
# df_sub_metering_2.describe()
df_group_3.describe()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_3,dayofweek,month,dayofyear,Hour,Minute
count,86996.000000,86996.000000,86996.000000,86996.000000,86996.000000,90767.000000,90767.000000,90767.000000,90767.000000,90767.000000
mean,2.182119,0.137726,237.769858,9.210244,17.219355,3.007007,4.504512,120.239305,13.261626,29.494078
std,1.157982,0.123673,3.554284,4.994707,1.344115,2.038492,2.979774,86.091500,5.973373,17.355877
min,0.160000,0.000000,223.490000,0.800000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,1.380000,0.052000,235.300000,5.800000,17.000000,1.000000,2.000000,50.000000,9.000000,14.000000
50%,1.652000,0.108000,238.130000,7.000000,17.000000,3.000000,4.000000,111.000000,13.000000,30.000000
75%,2.696000,0.208000,240.330000,11.400000,18.000000,5.000000,6.000000,168.000000,19.000000,45.000000
max,10.670000,1.148000,249.760000,46.400000,20.000000,6.000000,12.000000,340.000000,23.000000,59.000000


df_group_3

In [ ]:
df_group_3.plot(kind='scatter', x='Sub_metering_3', y='Global_active_power', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

df_group_2.plot(kind='scatter', x='Sub_metering_2', y='Global_active_power', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

df_group_1.plot(kind='scatter', x='Sub_metering_1', y='Global_active_power', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
df_group_3['Voltage'].plot(kind='hist', bins=20, title='Voltage')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
df_group_3['Global_active_power'].plot(kind='hist', bins=20, title='Voltage')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
df_group_3.plot(figsize=(7,6))

In [ ]:
df.shape
print(df.isnull().sum())
df = df.dropna(subset=['Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3'])
df

In [ ]:
#We will be working with the forecasting of Sub Metering 1 == temp as a measure of precision
temp = df['Sub_metering_1']
temp.plot()

In [ ]:
#Defining window size of forecast
#[1, 2, 3, 4, 5] --> [6] minute
def df_to_X_y(df, window_size=5):
  df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np)-window_size):
    row = [[a] for a in df_as_np[i:i+window_size]]
    X.append(row)
    label = df_as_np[i+window_size]
    y.append(label)
  return np.array(X), np.array(y)

In [ ]:
WINDOW_SIZE = 5
X1, y1 = df_to_X_y(temp, WINDOW_SIZE)
X1.shape, y1.shape

In [ ]:
#256864 total records
X_train1, y_train1 = X1[:180000], y1[:180000]
X_val1, y_val1 = X1[180000:220000], y1[180000:220000]
X_test1, y_test1 = X1[220000:], y1[220000:]
X_train1.shape, y_train1.shape, X_val1.shape, y_val1.shape, X_test1.shape, y_test1.shape

In [ ]:
#Preprocessing Data - scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train1 = scaler.fit_transform(X_train1.reshape(-1, X_train1.shape[-1])).reshape(X_train1.shape)
X_val1 = scaler.transform(X_val1.reshape(-1, X_val1.shape[-1])).reshape(X_val1.shape)
X_test1 = scaler.transform(X_test1.reshape(-1, X_test1.shape[-1])).reshape(X_test1.shape)

In [ ]:
#Making DateTime column as index
df['DateTime'] = df.index
df = df[['DateTime'] + [col for col in df.columns if col != 'DateTime']]

In [ ]:
#MODEL 1 : LSTM with ReLU
model1 = Sequential()
model1.add(InputLayer((5, 1)))
model1.add(LSTM(64))
model1.add(Dense(8, 'relu'))
model1.add(Dense(1, 'linear'))

model1.summary()

In [ ]:
cp1 = ModelCheckpoint('model1/', save_best_only=True)
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model1.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=10, callbacks=[cp1])

In [ ]:
#Predictions

def plot_predictions1(model, X, y, start=0, end=100):
  predictions = model.predict(X).flatten()
  df = pd.DataFrame(data={'Predictions':predictions, 'Actuals':y})
  plt.plot(df['Predictions'][start:end])
  plt.plot(df['Actuals'][start:end])
  return df, mse(y, predictions)

train_predictions = model1.predict(X_train1).flatten()
train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':y_train1})
train_results
val_predictions = model1.predict(X_val1).flatten()
val_results = pd.DataFrame(data={'Val Predictions':val_predictions, 'Actuals':y_val1})
val_results

In [ ]:
plot_predictions1(model1, X_test1, y_test1)

In [ ]:
#MODEL 2: 1D CNN 
model2 = Sequential()
model2.add(InputLayer((5, 1)))
model2.add(Conv1D(64, kernel_size=2))
model2.add(Flatten())
model2.add(Dense(8, 'relu'))
model2.add(Dense(1, 'linear'))

model2.summary()

In [ ]:
cp2= ModelCheckpoint('model2/', save_best_only=True)
model2.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model2.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=10, callbacks=[cp2])

In [ ]:
plot_predictions1(model2, X_test1, y_test1)

In [ ]:
#MODEL 3 : GRU
model3 = Sequential()
model3.add(InputLayer((5, 1)))
model3.add(GRU(64))
model3.add(Dense(8, 'relu'))
model3.add(Dense(1, 'linear'))
model3.summary()

In [ ]:
cp3 = ModelCheckpoint('model3/', save_best_only=True)
model3.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
plot_predictions1(model3, X_test1, y_test1)

In [ ]:
#Creating precise timestamps
temp_df = pd.DataFrame({'Sub_1':temp})
temp_df['Seconds'] = temp_df.index.map(pd.Timestamp.timestamp)
temp_df

day = 60*60*24
year = 365.2425*day

temp_df['Day sin'] = np.sin(temp_df['Seconds'] * (2* np.pi / day))
temp_df['Day cos'] = np.cos(temp_df['Seconds'] * (2 * np.pi / day))
temp_df['Year sin'] = np.sin(temp_df['Seconds'] * (2 * np.pi / year))
temp_df['Year cos'] = np.cos(temp_df['Seconds'] * (2 * np.pi / year))
temp_df.head()

In [ ]:
temp_df = temp_df.drop('Seconds', axis=1)
temp_df.head()

In [ ]:
def df_to_X_y2(df, window_size=6):
  df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np)-window_size):
    row = [r for r in df_as_np[i:i+window_size]]
    X.append(row)
    label = df_as_np[i+window_size][0]
    y.append(label)
  return np.array(X), np.array(y)

In [ ]:
X2, y2 = df_to_X_y2(temp_df)
X2.shape, y2.shape

In [ ]:
X_train2, y_train2 = X2[:180000], y2[:180000]
X_val2, y_val2 = X2[180000:220000], y2[180000:220000]
X_test2, y_test2 = X2[220000:], y2[220000:]
X_train2.shape, y_train2.shape, X_val2.shape, y_val2.shape, X_test2.shape, y_test2.shape

In [ ]:
temp_training_mean = np.mean(X_train2[:, :, 0])
temp_training_std = np.std(X_train2[:, :, 0])

def preprocess(X):
  X[:, :, 0] = (X[:, :, 0] - temp_training_mean) / temp_training_std
  return X

In [ ]:
preprocess(X_train2)
preprocess(X_val2)
preprocess(X_test2)

In [ ]:
#MODEL 4: LSTM with standardisation 
model4 = Sequential()
model4.add(InputLayer((6, 5)))
model4.add(LSTM(64))
model4.add(Dense(8, 'relu'))
model4.add(Dense(1, 'linear'))

model4.summary()

In [ ]:
cp4 = ModelCheckpoint('model4/', save_best_only=True)
model4.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model4.fit(X_train2, y_train2, validation_data=(X_val2, y_val2), epochs=10, callbacks=[cp4])

In [ ]:
plot_predictions1(model4, X_test2, y_test2)